In [1]:
%load_ext autoreload
%autoreload 2

from pathlib import Path
import shutil
import os
import logging
import sys
sys.path.append('..')

from textblob import TextBlob
from pprint import pprint
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler

from transformers import AutoModelForSequenceClassification

from finbert.finbert import *
import finbert.utils as tools
from finbert.custom_finbert import CustomFinBert

project_dir = Path.cwd().parent
pd.set_option('max_colwidth', -1)

logging.basicConfig(format = '%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
                    datefmt = '%m/%d/%Y %H:%M:%S',
                    level = logging.ERROR)

In [2]:
standard_dir = project_dir.parent.parent

# ! TODO: PUT THE PREPROCESSING CODE HERE AND STORE PROCESSED DF

# Prepare model

In [3]:
lm_path = project_dir/'models'/'language_model'/'finbertTRC2'
final_model_dir = project_dir/'models'/'classification_model_lr_1e-4'
data_dir = standard_dir/"Data"
# classification_data_path= standard_dir/'Data'/'sentiment_data'

## Configure training parameters

In [4]:
# Clean the cl_path
try:
    shutil.rmtree(final_model_dir) 
except:
    pass

## load the pretrained language model trained on Reuters TRC2
bertmodel = AutoModelForSequenceClassification.from_pretrained(
    lm_path, cache_dir=None, num_labels=30)

Some weights of the model checkpoint at /media/student/HDD 2/Michelle/MLS/advanced/finBERT_transaction_classification/models/language_model/finbertTRC2 were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
S

In [5]:
custom_finbert = CustomFinBert(bertmodel, 38, 30, hidden_dims=[])

In [7]:
config = Config(
    data_dir=data_dir,
    model_dir=final_model_dir,
    bert_model=custom_finbert,
    num_train_epochs=10,
    max_seq_length=48,
    train_batch_size=32,
    learning_rate=1e-4,
    output_mode="classification",
    warm_up_proportion=0.2,
    local_rank=-1,
    discriminate=True,
    gradual_unfreeze=False,
    numeric_params=38,
    label_colname="category"
)

In [8]:
finbert = FinBert(config, transaction_classification=True)

In [9]:
y = pd.read_csv(standard_dir/"Data"/"eval.csv")["category"]

In [10]:
finbert.prepare_model(label_list=y.unique().tolist())

07/18/2024 15:48:21 - INFO - finbert.finbert -   device: cuda n_gpu: 2, distributed training: False, 16-bits training: False


In [11]:
train_examples, numeric_feats = finbert.get_data("train")

In [12]:
model = finbert.create_the_model()

In [13]:
trained_model = finbert.train(train_examples = train_examples, model = model, train_numeric_feats=numeric_feats)

07/18/2024 15:48:30 - INFO - finbert.utils -   *** Example ***
07/18/2024 15:48:30 - INFO - finbert.utils -   guid: 0
07/18/2024 15:48:30 - INFO - finbert.utils -   tokens: [CLS] ac ##h withdrawal am ##mar ##yse hem ##ant ##ne ##x ##press transfer [SEP]
07/18/2024 15:48:30 - INFO - finbert.utils -   input_ids: 101 9353 2232 10534 2572 7849 23274 19610 4630 2638 2595 20110 4651 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
07/18/2024 15:48:30 - INFO - finbert.utils -   attention_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
07/18/2024 15:48:30 - INFO - finbert.utils -   token_type_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
07/18/2024 15:48:30 - INFO - finbert.utils -   label: Transfer Debit (id = 2)
07/18/2024 15:48:42 - INFO - finbert.finbert -   ***** Loading data *****
07/18/2024 15:48:42 - INFO - finbert.finbert -     Num examples = 160391
07/18

Iteration:   0%|          | 0/5013 [00:00<?, ?it/s]

(Iteration 0) Training loss: 3.4555718898773193
(Iteration 100) Training loss: 3.219036102294922
(Iteration 200) Training loss: 3.37111234664917
(Iteration 300) Training loss: 3.6875386238098145
(Iteration 400) Training loss: 3.483788013458252
(Iteration 500) Training loss: 2.8606741428375244
(Iteration 600) Training loss: 2.557075262069702
(Iteration 700) Training loss: 2.3294999599456787
(Iteration 800) Training loss: 1.8645200729370117
(Iteration 900) Training loss: 1.7264622449874878
(Iteration 1000) Training loss: 1.751303791999817
(Iteration 1100) Training loss: 2.0255682468414307
(Iteration 1200) Training loss: 1.5908957719802856
(Iteration 1300) Training loss: 1.3990073204040527
(Iteration 1400) Training loss: 1.4846954345703125
(Iteration 1500) Training loss: 1.2271440029144287
(Iteration 1600) Training loss: 1.1639950275421143
(Iteration 1700) Training loss: 0.8918479681015015
(Iteration 1800) Training loss: 0.6752336621284485
(Iteration 1900) Training loss: 1.021091341972351

07/18/2024 16:08:52 - INFO - finbert.utils -   *** Example ***
07/18/2024 16:08:52 - INFO - finbert.utils -   guid: 0
07/18/2024 16:08:52 - INFO - finbert.utils -   tokens: [CLS] rt ##p received money ##lion ins ##ta ##cas ##h 48 ##8 [SEP]
07/18/2024 16:08:52 - INFO - finbert.utils -   input_ids: 101 19387 2361 2363 2769 18964 16021 2696 15671 2232 4466 2620 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
07/18/2024 16:08:52 - INFO - finbert.utils -   attention_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
07/18/2024 16:08:52 - INFO - finbert.utils -   token_type_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
07/18/2024 16:08:52 - INFO - finbert.utils -   label: Loans (id = 0)
07/18/2024 16:08:57 - INFO - finbert.finbert -   ***** Loading data *****
07/18/2024 16:08:57 - INFO - finbert.finbert -     Num examples = 68739
07/18/2024 16:08:57 - INFO - fin

Validating:   0%|          | 0/2149 [00:00<?, ?it/s]

...

(Iteration 0) Training loss: 0.05954249948263168
(Iteration 100) Training loss: 0.08362004160881042
(Iteration 200) Training loss: 0.6835175156593323
(Iteration 300) Training loss: 0.1416778266429901
(Iteration 400) Training loss: 0.3615134656429291
(Iteration 500) Training loss: 0.17148441076278687
(Iteration 600) Training loss: 0.15235929191112518
(Iteration 700) Training loss: 0.09528184682130814
(Iteration 800) Training loss: 0.06382440775632858
(Iteration 900) Training loss: 0.07699044048786163
(Iteration 1000) Training loss: 0.05135718733072281
(Iteration 1100) Training loss: 0.0718797966837883
(Iteration 1200) Training loss: 0.14104001224040985
(Iteration 1300) Training loss: 0.09840114414691925
(Iteration 1400) Training loss: 0.04215167835354805
(Iteration 1500) Training loss: 0.036543816328048706
(Iteration 1600) Training loss: 0.010139521211385727
(Iteration 1700) Training loss: 0.10808117687702179
(Iteration 1800) Training loss: 0.04660505801439285
(Iteration 1900) Training 

07/18/2024 19:32:44 - INFO - finbert.utils -   *** Example ***
07/18/2024 19:32:44 - INFO - finbert.utils -   guid: 0
07/18/2024 19:32:44 - INFO - finbert.utils -   tokens: [CLS] rt ##p received money ##lion ins ##ta ##cas ##h 48 ##8 [SEP]
07/18/2024 19:32:44 - INFO - finbert.utils -   input_ids: 101 19387 2361 2363 2769 18964 16021 2696 15671 2232 4466 2620 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
07/18/2024 19:32:44 - INFO - finbert.utils -   attention_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
07/18/2024 19:32:44 - INFO - finbert.utils -   token_type_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
07/18/2024 19:32:44 - INFO - finbert.utils -   label: Loans (id = 0)
07/18/2024 19:32:49 - INFO - finbert.finbert -   ***** Loading data *****
07/18/2024 19:32:49 - INFO - finbert.finbert -     Num examples = 68739
07/18/2024 19:32:49 - INFO - fin

Validating:   0%|          | 0/2149 [00:00<?, ?it/s]

Validation losses: [0.5799220393079416, 0.5043733746258445, 0.41001141623374143, 0.396769596618917, 0.42151122965939647, 0.3938752062074414, 0.37931189097748164, 0.38214523738286754, 0.3901790154324485, 0.39054488360537487]
Validation Accuracy: 0.9071


Epoch: 100%|██████████| 10/10 [3:46:31<00:00, 1359.11s/it]


In [14]:
import pickle
with open("./trained_model_4.pkl", "wb") as f:
    pickle.dump(trained_model, f)

In [21]:
test_examples, test_numeric_feats = finbert.get_data("eval")

In [22]:
results = finbert.evaluate(trained_model, test_examples, test_numeric_feats)

07/18/2024 20:04:41 - INFO - finbert.utils -   *** Example ***
07/18/2024 20:04:41 - INFO - finbert.utils -   guid: 0
07/18/2024 20:04:41 - INFO - finbert.utils -   tokens: [CLS] rt ##p received money ##lion ins ##ta ##cas ##h 48 ##8 [SEP]
07/18/2024 20:04:41 - INFO - finbert.utils -   input_ids: 101 19387 2361 2363 2769 18964 16021 2696 15671 2232 4466 2620 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
07/18/2024 20:04:41 - INFO - finbert.utils -   attention_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
07/18/2024 20:04:41 - INFO - finbert.utils -   token_type_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
07/18/2024 20:04:41 - INFO - finbert.utils -   label: Loans (id = 0)


07/18/2024 20:04:46 - INFO - finbert.finbert -   ***** Loading data *****
07/18/2024 20:04:46 - INFO - finbert.finbert -     Num examples = 68739
07/18/2024 20:04:46 - INFO - finbert.finbert -     Batch size = 32
07/18/2024 20:04:46 - INFO - finbert.finbert -     Num steps = 21480
07/18/2024 20:04:46 - INFO - finbert.finbert -   ***** Running evaluation ***** 
07/18/2024 20:04:46 - INFO - finbert.finbert -     Num examples = 68739
07/18/2024 20:04:46 - INFO - finbert.finbert -     Batch size = 32


Testing:   0%|          | 0/2149 [00:00<?, ?it/s]

Evaluation loss: 0.2720436788037967


In [23]:
preds = results["predictions"].apply(lambda x: np.argmax(x, axis=0))

In [24]:
label_mapping = {i:label for i, label in enumerate(finbert.label_list)}

In [25]:
preds = [label_mapping[p] for p in preds]

In [26]:
labels = [label_mapping[l] for l in results["labels"]]

In [27]:
print(classification_report(labels, preds, digits=3))

/home/student/anaconda3/envs/mls/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


                            precision    recall  f1-score   support

                       ATM      0.996     0.999     0.998      1702
    Arts and Entertainment      0.921     0.975     0.947       119
                 Bank Fees      0.994     0.993     0.994      1940
             Check Deposit      0.940     1.000     0.969        63
  Clothing and Accessories      0.860     0.889     0.875       957
        Convenience Stores      0.721     0.711     0.716      5589
         Department Stores      0.685     0.844     0.756       601
     Digital Entertainment      0.946     0.932     0.939      1358
Food and Beverage Services      0.683     0.835     0.751       103
              Gas Stations      0.470     0.595     0.525      3876
  Gyms and Fitness Centers      0.679     0.905     0.776        21
                Healthcare      0.568     0.742     0.643        62
                 Insurance      0.959     0.888     0.922       526
                  Interest      1.000     1.000